# Real Time Translation of Streamed Text into German

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.StreamingContext._ 
import scala.io.Source

// the duration in seconds
val intervalDuration = 2

// val spark = SparkSession.builder.appName("Scala Automated Translation").getOrCreate()
// val sc = spark.sparkContext
// val ssc = new StreamingContext(sc, intervalDuration)

val conf = new SparkConf().setAppName("Scala Automated Translation").setMaster("local[*]")
val ssc = new StreamingContext(conf, Seconds(intervalDuration))

def getDictionary = {
  val url="http://www.june29.com/idp/files/German.txt"

  val s1 = Source.fromURL(url, "ISO-8859-1").mkString
  val s2 = s1.split("\r?\n")
  val s3 = s2.filter(line => !line.startsWith("#"))
  val s4 = s3.map(line => line.split("\t")).map(s => (s(0), s(1))).toMap
  s4
}

val dct = getDictionary

val hostname = "nc"
val port = 9999

// create a DStream that will connect to hostname:port
val lines = ssc.socketTextStream(hostname, port)

val trns = lines.map(line => line.split("\\s+").map(word => dct.getOrElse(word, word)).mkString(" ")).cache
  
trns.print()

ssc.start()             // starting the computation
ssc.awaitTermination()  // waiting for the computation to terminate